# Find nearest hospitals

For each LSOA, find its nearest hospital offering IVT and its nearest offering MT.

## Setup

In [2]:
import pandas as pd

## Import data files

### Travel time matrix

In [5]:
df_travel_matrix = pd.read_csv('./lsoa_travel_time_matrix_calibrated.csv')

df_travel_matrix.head()

,LSOA,B152TH,B714HJ,B95SS,BA13NG,BA214AT,BB23HH,BD96RJ,BH77DW,BL97TD,...,TN240LZ,TQ27AA,TR13LQ,TS198PE,TS43BW,WD180HB,WF14DG,WR51DD,WV100QP,YO318HE
0,Adur 001A,173.3,179.8,171.2,161.5,152.9,274.3,261.4,100.3,262.5,...,89.6,212.0,291.5,310.8,306.5,92.8,242.1,178.7,186.2,267.9
1,Adur 001B,173.3,179.8,172.3,161.5,152.9,274.3,261.4,99.2,263.6,...,90.6,212.0,290.4,311.9,306.5,92.8,242.1,178.7,186.2,267.9
2,Adur 001C,173.3,180.9,172.3,150.8,151.9,274.3,261.4,98.2,263.6,...,90.6,210.9,289.3,311.9,307.6,93.9,243.2,178.7,187.3,268.9
3,Adur 001D,173.3,180.9,172.3,161.5,152.9,274.3,261.4,99.2,263.6,...,90.6,212.0,290.4,311.9,307.6,93.9,243.2,178.7,187.3,268.9
4,Adur 001E,174.4,180.9,173.3,150.8,151.9,275.4,262.5,98.2,264.6,...,91.7,209.9,289.3,313.0,307.6,93.9,243.2,179.8,187.3,268.9


In [8]:
# List of LSOAs used:
LSOA11NM = df_travel_matrix['LSOA']

# List of hospital postcodes:
# (skip the first column which is the "LSOA" heading)
hospital_postcodes = df_travel_matrix.columns[1:]

### LSOA names and codes table

In [94]:
df_regions = pd.read_csv('./LSOA_regions.csv')

df_regions.head()

,LSOA11CD,LSOA11NM,CCG19CD,CCG19NM,STP19CD,STP19NM,LHB20CD,LHB20NM,LHB20NMW,LAD17CD,LAD17NM,SCN17CD,SCN17NM,RGN11CD,RGN11NM
0,E01031349,Adur 001A,E38000213,NHS Coastal West Sussex CCG,E54000033,Sussex and East Surrey,NaN,NaN,NaN,E07000223,Adur,E55000010,South East Coast,E12000008,South East
1,E01031350,Adur 001B,E38000213,NHS Coastal West Sussex CCG,E54000033,Sussex and East Surrey,NaN,NaN,NaN,E07000223,Adur,E55000010,South East Coast,E12000008,South East
2,E01031351,Adur 001C,E38000213,NHS Coastal West Sussex CCG,E54000033,Sussex and East Surrey,NaN,NaN,NaN,E07000223,Adur,E55000010,South East Coast,E12000008,South East
3,E01031352,Adur 001D,E38000213,NHS Coastal West Sussex CCG,E54000033,Sussex and East Surrey,NaN,NaN,NaN,E07000223,Adur,E55000010,South East Coast,E12000008,South East
4,E01031370,Adur 001E,E38000213,NHS Coastal West Sussex CCG,E54000033,Sussex and East Surrey,NaN,NaN,NaN,E07000223,Adur,E55000010,South East Coast,E12000008,South East


In [95]:
LSOA11NM_regions = df_regions['LSOA11NM']

Check whether this list is identical to the list in the other file:

In [99]:
(LSOA11NM_regions == LSOA11NM).all()

True

The above line should return `True`. So just use the LSOA code column exactly as it is.

In [100]:
LSOA11CD = df_regions['LSOA11CD']

### Hospital information

In [9]:
df_hospitals = pd.read_csv('../data/stroke_hospitals_2022.csv')

df_hospitals.head()

,Postcode,Hospital_name,Use_IVT,Use_MT,Use_MSU,Country,Strategic Clinical Network,Health Board / Trust,Stroke Team,SSNAP name,...,Thrombolysis,ivt_rate,Easting,Northing,long,lat,Neuroscience,30 England Thrombectomy Example,hospital_city,Notes
0,RM70AG,RM70AG,1,1,1,England,London SCN,Barking,Havering and Redbridge University Hospitals N...,Queens Hospital Romford HASU,...,117,11.9,551118,187780,0.179031,51.568647,1,0,Romford,NaN
1,E11BB,E11BB,1,1,1,England,London SCN,Barts Health NHS Trust,The Royal London Hospital,Royal London Hospital HASU,...,115,13.4,534829,181798,-0.058133,51.519018,1,1,Royal London,NaN
2,SW66SX,SW66SX,1,1,1,England,London SCN,Imperial College Healthcare NHS Trust,"Charing Cross Hospital, London",Charing Cross Hospital HASU,...,113,9.9,524226,176487,-0.212736,51.473717,1,1,Charing Cross,NaN
3,SE59RW,SE59RW,1,1,1,England,London SCN,King's College Hospital NHS Foundation Trust,"King's College Hospital, London",King's College Hospital HASU,...,124,15.0,532536,176228,-0.093251,51.469505,1,0,Kings College,NaN
4,BR68ND,BR68ND,1,0,0,England,London SCN,King's College Hospital NHS Foundation Trust,Princess Royal University Hospital,Princess Royal University Hospital HASU,...,113,13.3,543443,165032,0.059146,51.366243,0,0,Princess Royal,NaN


In [78]:
hospital_info_postcodes = df_hospitals['Postcode']
hospital_info_names = df_hospitals['Stroke Team']

In [79]:
# List of hospitals that use MT:
b = (df_hospitals['Use_IVT'] == 1) & (df_hospitals['Use_MT'] == 1)
# Taking the values to save later confusion over changed indices
hospital_info_postcodes_MT = df_hospitals['Stroke Team'][b].values
hospital_info_names_MT = df_hospitals['Postcode'][b].values

In [80]:
# Also find the hospitals that do not use MT 
# for easier dropping of column from the travel matrix.
n = (df_hospitals['Use_IVT'] == 1) & (df_hospitals['Use_MT'] == 0)
hospital_info_postcodes_no_MT_yes_IVT = df_hospitals['Postcode'][n].values

hospital_info_postcodes_no_MT_yes_IVT

array(['BR68ND', 'HA13UJ', 'DE223NE', 'NN15BD', 'NG174JL', 'LN25QY',
       'LE15WW', 'SS165NL', 'CO45JL', 'SG14AB', 'IP45PD', 'NR316LA',
       'LU40DZ', 'CM17ET', 'NR47UY', 'PE39GZ', 'PE304ET', 'SS00RY',
       'WD180HB', 'IP332QZ', 'DY12HQ', 'WV100QP', 'B714HJ', 'TF16TF',
       'WR51DD', 'HR12ER', 'CH21UL', 'CW14QJ', 'L355DR', 'CH495PE',
       'FY38NR', 'BB23HH', 'BL97TD', 'SK27JE', 'LA144LF', 'LA14RP',
       'SR47TP', 'DH15TW', 'CA27HY', 'TS198PE', 'NE236NZ', 'TS43BW',
       'BD96RJ', 'HX30PW', 'S445BL', 'DN25LT', 'WF14DG', 'DN157BH',
       'YO318HE', 'KT160PZ', 'DA28DA', 'CT13NG', 'BN212UD', 'KT187EG',
       'GU167UJ', 'ME169QQ', 'RH15RH', 'PO196SE', 'BN112DH', 'GL13NN',
       'SN36BB', 'EX314JB', 'TR13LQ', 'EX25DW', 'BA13NG', 'SP28BJ',
       'TA15DA', 'TQ27AA', 'BS234TQ', 'BA214AT', 'HP112TT', 'MK65LD',
       'RG15AN', 'DT12JY', 'SO225DG', 'PO305TG', 'PO63LY', 'BH77DW',
       'NP448YN', 'LL185UJ', 'LL137TD', 'LL572PW', 'CF479DT', 'CF311RQ',
       'SY231ER', 'SA148QF', 

## Find nearest hospital

In [81]:
df_travel_times = df_travel_matrix.drop('LSOA', axis=1)

df_travel_times.head()

,B152TH,B714HJ,B95SS,BA13NG,BA214AT,BB23HH,BD96RJ,BH77DW,BL97TD,BN112DH,...,TN240LZ,TQ27AA,TR13LQ,TS198PE,TS43BW,WD180HB,WF14DG,WR51DD,WV100QP,YO318HE
0,173.3,179.8,171.2,161.5,152.9,274.3,261.4,100.3,262.5,18.7,...,89.6,212.0,291.5,310.8,306.5,92.8,242.1,178.7,186.2,267.9
1,173.3,179.8,172.3,161.5,152.9,274.3,261.4,99.2,263.6,18.7,...,90.6,212.0,290.4,311.9,306.5,92.8,242.1,178.7,186.2,267.9
2,173.3,180.9,172.3,150.8,151.9,274.3,261.4,98.2,263.6,17.6,...,90.6,210.9,289.3,311.9,307.6,93.9,243.2,178.7,187.3,268.9
3,173.3,180.9,172.3,161.5,152.9,274.3,261.4,99.2,263.6,17.6,...,90.6,212.0,290.4,311.9,307.6,93.9,243.2,178.7,187.3,268.9
4,174.4,180.9,173.3,150.8,151.9,275.4,262.5,98.2,264.6,16.5,...,91.7,209.9,289.3,313.0,307.6,93.9,243.2,179.8,187.3,268.9


In [82]:
# Mimumum travel time for each LSOA:
lsoa_nearest_times = df_travel_times.min(axis=1)

In [83]:
# Index of the minimum travel time for each LSOA:
lsoa_nearest_postcodes = df_travel_times.idxmin(axis=1)

In [84]:
# Match these postcodes to hospital names:
lsoa_nearest_hospitals = lsoa_nearest_postcodes.copy()

for h, hospital_postcode in enumerate(hospital_info_postcodes.values):
    b = lsoa_nearest_postcodes == hospital_postcode
    lsoa_nearest_hospitals[b] = hospital_info_names.values[h]

lsoa_nearest_hospitals

0        Royal Sussex County Hospital, Brighton
1                             Worthing Hospital
2                             Worthing Hospital
3                             Worthing Hospital
4                             Worthing Hospital
                          ...                  
34747                             York Hospital
34748                             York Hospital
34749                             York Hospital
34750                             York Hospital
34751                             York Hospital
Length: 34752, dtype: object

## Find nearest hospital offering MT

In [85]:
# Drop the LSOA column and all columns of hospitals that do not use MT:
cols_to_drop = ['LSOA'] + hospital_info_postcodes_no_MT_yes_IVT.tolist()

df_travel_times_mt = df_travel_matrix.drop(cols_to_drop, axis=1)

df_travel_times_mt.head()

,B152TH,B95SS,BN25BE,BS105NB,BS28HW,CB20QQ,CF144XW,CT94AN,CV107DJ,CV22DX,...,RM70AG,S102JF,S752EP,SE59RW,SL24HL,SO166YD,ST46QG,SW170QT,SW66SX,TN240LZ
0,173.3,171.2,17.6,162.6,160.5,122.9,192.7,120.7,163.7,159.4,...,85.3,223.8,230.3,74.5,78.8,76.7,212.0,69.2,76.7,89.6
1,173.3,172.3,18.7,162.6,160.5,122.9,192.7,120.7,163.7,159.4,...,85.3,224.9,230.3,74.5,78.8,76.7,212.0,69.2,76.7,90.6
2,173.3,172.3,19.8,162.6,161.5,123.9,192.7,121.8,163.7,160.5,...,86.4,224.9,231.3,74.5,79.9,75.6,212.0,70.2,77.8,90.6
3,173.3,172.3,19.8,162.6,161.5,122.9,192.7,120.7,163.7,159.4,...,85.3,224.9,231.3,74.5,79.9,75.6,212.0,70.2,77.8,90.6
4,174.4,173.3,19.8,163.7,161.5,123.9,193.8,121.8,164.8,160.5,...,86.4,226.0,231.3,75.6,79.9,74.5,213.1,70.2,77.8,91.7


In [86]:
# Mimumum travel time for each LSOA:
lsoa_nearest_mt_times = df_travel_times_mt.min(axis=1)

In [87]:
# Index of the minimum travel time for each LSOA:
lsoa_nearest_mt_postcodes = df_travel_times_mt.idxmin(axis=1)

In [88]:
# Match these postcodes to hospital names:
lsoa_nearest_mt_hospitals = lsoa_nearest_mt_postcodes.copy()

for h, hospital_postcode in enumerate(hospital_info_postcodes.values):
    b = lsoa_nearest_mt_postcodes == hospital_postcode
    lsoa_nearest_mt_hospitals[b] = hospital_info_names.values[h]

lsoa_nearest_mt_hospitals

0        Royal Sussex County Hospital, Brighton
1        Royal Sussex County Hospital, Brighton
2        Royal Sussex County Hospital, Brighton
3        Royal Sussex County Hospital, Brighton
4        Royal Sussex County Hospital, Brighton
                          ...                  
34747                   Leeds General Infirmary
34748                   Leeds General Infirmary
34749                   Leeds General Infirmary
34750                   Leeds General Infirmary
34751                   Leeds General Infirmary
Length: 34752, dtype: object

## Combined dataframe

In [101]:
# Combine LSOAs and their nearest hospitals into one dataframe:
df_lsoa_hospital = pd.concat((
    LSOA11CD,
    LSOA11NM,
    lsoa_nearest_postcodes,
    lsoa_nearest_hospitals,
    lsoa_nearest_times,
    lsoa_nearest_mt_postcodes,
    lsoa_nearest_mt_hospitals,
    lsoa_nearest_mt_times,
    ), axis=1)

# Rename column headings that didn't have names before:
new_col_names = [
    'Nearest hospital postcode',
    'Nearest hospital name',
    'Time (mins) to nearest hospital',
    'Nearest MT hospital postcode',
    'Nearest MT hospital name',
    'Time (mins) to nearest MT hospital'
    ]
for i, new_name in enumerate(new_col_names):
    df_lsoa_hospital.rename({i:new_name}, axis=1, inplace=True)

df_lsoa_hospital.head()

,LSOA11CD,LSOA,Nearest hospital postcode,Nearest hospital name,Time (mins) to nearest hospital,Nearest MT hospital postcode,Nearest MT hospital name,Time (mins) to nearest MT hospital
0,E01031349,Adur 001A,BN25BE,"Royal Sussex County Hospital, Brighton",17.6,BN25BE,"Royal Sussex County Hospital, Brighton",17.6
1,E01031350,Adur 001B,BN112DH,Worthing Hospital,18.7,BN25BE,"Royal Sussex County Hospital, Brighton",18.7
2,E01031351,Adur 001C,BN112DH,Worthing Hospital,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",19.8
3,E01031352,Adur 001D,BN112DH,Worthing Hospital,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",19.8
4,E01031370,Adur 001E,BN112DH,Worthing Hospital,16.5,BN25BE,"Royal Sussex County Hospital, Brighton",19.8


Save this dataframe to file:

In [102]:
df_lsoa_hospital.to_csv('lsoa_nearest_hospitals.csv', index=False)